In [1]:
%pylab

Using matplotlib backend: WXAgg
Populating the interactive namespace from numpy and matplotlib


In [10]:
import os
flynum = 487
exp_name = 'step_ptch_roll'
dir_files = os.listdir('/media/FlyDataC/FlyDB/Fly0'+str(flynum))
abf_file = [x for x in dir_files if '.abf' in x][0]
tiff_file_base = 'T2_trial1_ptch_roll'
init_txt = """
def initfly_%s(self,flynum):
    fly_db = self.fly_db
    fly_db.create_group(flynum)
    fly_record =fly_db[flynum]
    fly_record.create_group('experiments')
    fly_record['experiments'].create_group('%s')
    fly_record['experiments']['%s']['axon_file_names'] = ['%s']
    fly_record['experiments']['%s']['tiff_file_names'] = ['/%s/%s_MMStack.ome.tif']
"""%(flynum,
     exp_name,
     exp_name,abf_file,
     exp_name,tiff_file_base,tiff_file_base)
print init_txt

OSError: [Errno 2] No such file or directory: '/media/FlyDataC/FlyDB/Fly0487'

In [1]:
import flylib
import db_access as dba
fly_db = dba.get_db()

In [2]:
#test_group = [241,242,243,244,245,246,247]
#test_group = [241,242,243,244]
test_group = range(478,487)

In [13]:
initializer = dba.InitDB()
[initializer[fn] for fn in test_group]
initializer.fly_db.flush()
initializer.fly_db.close()

<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>


In [3]:
swarm = flylib.Squadron(fly_db,test_group)
for fly in swarm.flies:
    for expmnt in fly.experiments.values():
        try:
            expmnt = fly.experiments['step_ptch_roll']
            print expmnt.exp_record['axon_file_names'].value
            expmnt.import_axon_data()
            fly_db.flush()
            expmnt.import_tiff_data()
            fly_db.flush()
            expmnt.calc_framebase()
            fly_db.flush()
        except Exception, er:
            print er
        #expmnt.calc_framebase()
        #fly_db.flush()

['T2_trial1_ptch_roll_15430001.abf']
['T2_trial1_ptch_roll_15430002.abf']
['T2_trial1_ptch_roll_15430003.abf']
['T2_trial1_ptch_roll_15430005.abf']
['T2_trial1_ptch_roll_15501000.abf']
['T2_trial1_ptch_roll_15501001.abf']
['T2_trial1_ptch_roll_15501002.abf']
['T2_trial1_ptch_roll_15501003.abf']
['T2_trial1_ptch_roll_15501004.abf']


/media/FlyDataC/flight-muscles/src/flylib.py:616: RuntimeWarning: divide by zero encountered in divide
  update_dset(self.exp_record['axon_data'],'wb_frequency',1.0/period)


In [6]:
expmnt = fly.experiments.values()[0]

In [21]:
import numpy as np
data = np.array(expmnt.exp_record['tiff_data']['axon_framebase']['StimCond'])

In [45]:
dta = np.around((data-1)/9*12)

In [66]:
[plot(expmnt.exp_record['tiff_data']['axon_framebase']['Xpos'][idx[0]:idx[-1]]) for idx in idxs if dta[idx[10]] == 6]

[[<matplotlib.lines.Line2D at 0xe0eaa10>],

In [52]:
idxs = flylib.idx_by_thresh(dta,0)

In [51]:
shape(idx)

(88,)

In [8]:
plot(expmnt.exp_record['tiff_data']['axon_framebase']['wb_frequency'])

In [6]:
%pylab

Using matplotlib backend: WXAgg
Populating the interactive namespace from numpy and matplotlib


In [53]:
fly = swarm.flies[3]

In [54]:
expmnt = fly.experiments.values()[0]

In [55]:
expmnt.exp_record['tiff_data']['axon_framebase'].keys()

[u'AMsysCh1',
 u'CamSync',
 u'CamTrig',
 u'DAC2',
 u'LeftWing',
 u'Ph0',
 u'Ph1',
 u'Ph2',
 u'Ph3',
 u'Photostim',
 u'RightWing',
 u'StimCond',
 u'WBSync',
 u'Xpos',
 u'Ypos',
 u'times']

In [9]:
plot(expmnt.exp_record['tiff_data']['axon_framebase']['times'],expmnt.exp_record['tiff_data']['axon_framebase']['Photostim'])

In [51]:
plot(expmnt.exp_record['axon_data']['times'][:3000000],expmnt.exp_record['axon_data']['WBSync'][:3000000])

In [58]:
def calc_wb_period():
    wb_trigs = np.diff((np.array(expmnt.exp_record['axon_data']['WBSync']) > 1).astype(int))>0
    wb_trigs = np.hstack((wb_trigs,np.array([0])))
    wb_times = np.array(expmnt.exp_record['axon_data']['times'])
    trigidx = where(wb_trigs)[0]
    idx_intervals = vstack((hstack(([0],trigidx[:-1])),trigidx))
    time_intervals = vstack((wb_times[idx_intervals[0,:]],wb_times[idx_intervals[1,:]]))
    wb_periods = squeeze(diff(time_intervals,axis = 0 ))
    period_sig = zeros_like(wb_trigs).astype(float)
    for interval,period in zip(idx_intervals[:,:].T,wb_periods):
        period_sig[interval[0]:interval[1]] = period
    return period_sig

In [100]:
period = calc_wb_period()

In [60]:
plot(1/psig[::100])

-c:1: RuntimeWarning: divide by zero encountered in divide


In [132]:
fly = swarm.flies[8]
expmnt = fly.experiments.values()[0]
sigs = expmnt.exp_record['axon_data']
ax_dt = sigs['times'][1]-sigs['times'][0]
#block to get stim time during exposure
strobe = np.array(sigs['Photostim']) > 1
wmask = ((np.array(sigs['LeftWing']) + 
          np.array(sigs['RightWing']))/2.0) < 1.0
fmask = np.array(sigs['CamSync'])> 1
strobe = strobe & wmask & fmask
exposures = flb.idx_by_thresh(np.array(sigs['CamSync']),thresh = 1)
dsamp1 = np.array([np.sum(strobe[ex])*ax_dt for ex in exposures])

In [133]:
plot(dsamp1)

In [131]:
fly.fly_num

309

In [105]:
expmnt.exp_record['tiff_data']['axon_framebase'].keys()

[u'AMsysCh1',
 u'CamSync',
 u'CamTrig',
 u'DAC2',
 u'LeftWing',
 u'Ph0',
 u'Ph1',
 u'Ph2',
 u'Ph3',
 u'Photostim',
 u'RightWing',
 u'StimCond',
 u'WBSync',
 u'Xpos',
 u'Ypos',
 u'times']

2.0000000000000002e-05

In [75]:
plot(wb_times[:10000000])

In [8]:
fly = swarm.flies[3]
exp = fly.experiments['img_pattern_test_t2_lightoff']
sequences = exp.exp_record['sequences'].values()
sig = np.array(exp.exp_record['axon_data']['Ph1'])
for seq in sequences:
    if seq['epoch_name'].value == 'static_test_roll_90.mat':
        subplot(3,1,2)
        ep = np.array(seq['epoch'])
        plot(sig[ep],color = 'r',alpha = 0.5)
    if seq['epoch_name'].value == 'static_test_roll_270.mat':
        subplot(3,1,2)
        ep = np.array(seq['epoch'])
        plot(sig[ep],color = 'b',alpha = 0.5)
    if seq['epoch_name'].value == 'static_test_yaw_90.mat':
        subplot(3,1,3)
        ep = np.array(seq['epoch'])
        plot(sig[ep],color = 'r',alpha = 0.5)
    if seq['epoch_name'].value == 'static_test_yaw_270.mat':
        subplot(3,1,3)
        ep = np.array(seq['epoch'])
        plot(sig[ep],color = 'b',alpha = 0.5)

In [16]:
d2 = np.array(expmnt.exp_record['axon_data']['DAC2'])
for key in sorted([int(x) for x in expmnt.exp_record['sequences'].keys()])[0:12]:
    print expmnt.exp_record['sequences'][str(key)]['epoch_name'].value
    ep = np.array(expmnt.exp_record['sequences'][str(key)]['epoch'][::1000])
    plot(d2[ep])

static_test_90.mat
static_test_90.mat
static_test_90.mat
static_test_90.mat
static_test_90.mat
static_test_90.mat
static_test_90.mat
static_test_90.mat
static_test_90.mat
static_test_90.mat
static_test_90.mat
static_test_90.mat


In [17]:
fly_db.flush()

In [18]:
fly_db.close()

<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>


In [14]:
set([99, 50, 17, 66, 33, 83, 99, 50, 17, 66, 83, 33, 33, 66, 17, 99, 50, 83, 66, 99, 17, 33, 83, 50, 66, 50, 99, 33, 17, 83, 66, 99, 50, 83, 17, 33, 33, 50, 83, 66, 17, 99, 33, 17, 83, 50, 99, 66, 66, 99, 17, 50, 83, 33, 33, 99, 17, 83, 66, 50, 83, 50, 17, 66, 33, 99, 66, 50, 99, 17, 33, 83])

{17, 33, 50, 66, 83, 99}

In [7]:
fly_db.close()

<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>
<Closed HDF5 file>


In [2]:
flynum = str(130)
fly_db.create_group(flynum)
fly_record = fly_db[flynum]
fly_record['flynum'] = flynum
fly_record.create_group('experiments')
fly_record['experiments'].create_group('lr_blob_expansion')
#fly_record['experiments'].create_group('b1_azm_expansion_tuning')
fly_record['experiments']['lr_blob_expansion']['photron_seq_nums'] = [1,2,3,4,5,6]
fly_record['experiments']['lr_blob_expansion']['axon_file_names'] = ['fly01_lr_blob_expansion_14506005.abf']
fly_record['experiments']['lr_blob_expansion']['photron_date_string'] = ['20140506']
fly_record['experiments']['lr_blob_expansion']['kine_filename'] = ['WBkin.mat']
fly_record['experiments']['lr_blob_expansion']['solution_format_string'] = ['20140506_S%04d/']
fly_record['experiments']['lr_blob_expansion']['photron_frame_rate_Hz'] = 6000
fly_record['experiments']['lr_blob_expansion']['Ypos_trial_volts'] = np.linspace(1,10,12)
fly_record['experiments']['lr_blob_expansion']['Ypos_trial_vals'] = np.concatenate(([np.nan],np.arange(0,12)*30))
fly_record['experiments']['lr_blob_expansion']['AMsysCh1_ID'] = 'i1'
fly_record['experiments']['lr_blob_expansion']['AMsysCh1_side'] = 'r'
fly_record['experiments']['lr_blob_expansion'].create_group('sequences')

<HDF5 group "/130/experiments/lr_blob_expansion/sequences" (0 members)>

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd')]

In [3]:
#ephys_group = [111,112,114,115,116,117,118,122,123,124,125]
i1_group = [130]
swarm = flylib.Squadron(fly_db,i1_group)
for fly in swarm.flies:
    lrb = fly.experiments['lr_blob_expansion']
    print 'importing data for fly%s'%fly.fly_num
    lrb.import_axon_data()
    lrb.import_sequence_data()
    print 'syncing data'
    lrb.sync_sequences()
    print lrb.sequences.keys()
    for seq_key in lrb.sequences.keys():
        seq = lrb.sequences[seq_key]
        print 'initializing seq %s'%(seq.seq_num)
        seq.calc_kine_phases(fband = (180,350))
        seq.calc_wb_mtrx()
        seq.calc_stim_frame(x_thresh = 2.5)
        fly_db.flush()

importing data for fly130
syncing data
[1, 2, 3, 4, 5, 6]
initializing seq 1
initializing seq 2
initializing seq 3
initializing seq 4
initializing seq 5
initializing seq 6


flylib.py:230: UserWarning: problem extracting idxs from camera_sync_signal for sequence 1 using even spaced idx's over the camera epoch instead
  warnings.warn("problem extracting idxs from camera_sync_signal for sequence %s using even spaced idx's over the camera epoch instead"%(snum))


In [5]:
imaging_group = [151,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169]
swarm = flylib.Squadron(fly_db,imaging_group)
for fly in swarm.flies:
    img_star = fly.experiments['img_starfield_t2_rep1']
    print 'importing data for fly%s'%fly.fly_num
    img_star.import_axon_data()
    img_star.import_tiff_data()
    fly_db.flush()

importing data for fly151
importing data for fly153
importing data for fly154
importing data for fly155
importing data for fly156
importing data for fly157
importing data for fly158
importing data for fly159
importing data for fly160
importing data for fly161
importing data for fly162
importing data for fly163
importing data for fly164
importing data for fly165
importing data for fly166
importing data for fly167
importing data for fly168
importing data for fly169


tifffile.py:2170: UserWarning: failed to import _tifffile.decodepackbits
  warnings.warn("failed to import %s" % module_function)
tifffile.py:2170: UserWarning: failed to import _tifffile.decodelzw
  warnings.warn("failed to import %s" % module_function)
tifffile.py:2170: UserWarning: failed to import _tifffile.unpackints
  warnings.warn("failed to import %s" % module_function)
